In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from pathlib import Path
import time
import joblib
import os

In [33]:
# Carregar os datasets

path = Path(os.getcwd())
CurrentFolder = path
MasterFile_Loadshape1 = CurrentFolder / 'objs' / '13Bus' / 'Datasets' / 'Loadshape1' / 'dataset_anual_8760h_11h.csv'
MasterFile_Loadshape2 = CurrentFolder / 'objs' / '13Bus' / 'Datasets' / 'Loadshape2' / 'dataset_anual_0h_8760h_8760h.csv'
data_Loadshape1 = pd.read_csv(MasterFile_Loadshape1)
data_Loadshape2 = pd.read_csv(MasterFile_Loadshape2)

             hora nome_barra       base  Geracao  Fase_0  Fase_1  Fase_2  \
0          hora_0        650   2.401777      NaN       1       1       1   
1          hora_0       rg60   2.401777      NaN       1       1       1   
2          hora_0        633   2.401777      NaN       1       1       1   
3          hora_0        634   0.277128      NaN       1       1       1   
4          hora_0        671   2.401777      NaN       1       1       1   
...           ...        ...        ...      ...     ...     ...     ...   
140155  hora_8759        670   2.401777      NaN       1       1       1   
140156  hora_8759        632   2.401777      NaN       1       1       1   
140157  hora_8759        680   2.401777      NaN       1       1       1   
140158  hora_8759        684   2.401777      NaN       1       0       1   
140159  hora_8759  sourcebus  66.395281      1.0       1       1       1   

        inj_pot_at_0  inj_pot_at_1  inj_pot_at_2  ...  ang_tensao_estimado_2  \
0      

In [10]:
# Salvar as colunas de "hora" e "nome_barra" para adicionar na saída final
hora = data_Loadshape1['hora']
nome_barra = data_Loadshape1['nome_barra']
base = data_Loadshape1['base']
geracao = data_Loadshape1['Geracao']

In [37]:
# Selecionar as features e os targets
# Entradas: Potência injetada, tensão, fases conectadas
features = data_Loadshape1[['Geracao',
                 'Fase_0', 'Fase_1', 'Fase_2',
                 'inj_pot_at_0', 'inj_pot_at_1', 'inj_pot_at_2', 
                 'inj_pot_rat_0', 'inj_pot_rat_1', 'inj_pot_rat_2',
                 'tensao_0', 'tensao_1', 'tensao_2']]

# Saídas: Ângulos de tensão estimados e tensões estimadas
targets = data_Loadshape1[['ang_tensao_estimado_0', 'ang_tensao_estimado_1', 'ang_tensao_estimado_2',
                'tensao_estimada_0', 'tensao_estimada_1', 'tensao_estimada_2']]


# Tratar valores faltantes (opção: remover ou preencher)
Entrada = features.fillna(value=0)
Saida = targets.fillna(value=0)

#print(Entrada)

#print(Saida)

In [12]:
# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(Entrada, Saida, test_size=0.2, random_state=42)

# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criando e treinando o modelo
inicio_treinamento = time.time()
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)
fim_treinamento = time.time()

print(f'IA treinada em {fim_treinamento-inicio_treinamento}s')

IA treinada em 75.35995411872864s


In [28]:
# Configuração da validação cruzada
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10-fold cross-validation

# Avaliar o modelo com validação cruzada
mse_scores = cross_val_score(model, Entrada, Saida, 
                             cv=kf, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(model, Entrada, Saida, 
                            cv=kf, scoring='r2')

# Como 'neg_mean_squared_error' retorna valores negativos, convertemos para positivo
mse_scores = -mse_scores

# Exibir resultados
print(f"Mean Squared Error (média): {np.mean(mse_scores)}")
print(f"R-squared Score (média): {np.mean(r2_scores)}")

Mean Squared Error (média): 0.045738331905239064
R-squared Score (média): 0.9521067862862405


In [29]:
print(mse_scores)
print(r2_scores)

[0.05048483 0.1067314  0.0363362  0.05810335 0.0745652  0.02165994
 0.07385065 0.01325231 0.01215736 0.01024207]
[0.94689004 0.90763465 0.959716   0.93643663 0.93362905 0.96195498
 0.93443735 0.97338679 0.97888457 0.98809779]
